In [ ]:
#!pip install httpx urllib3 python-dotenv

# API Functional Tests and Examples

This notebook is a collection of functional tests and examples for the API. It is not meant to be run as a whole, but rather to be used as a reference for the API. You'll need a .env file with the following variables:

```bash
API_KEY=your_api_key
API_SECRET=your_api_secret
```

You can also set the environment variables directly in the notebook.

The API documentation can be found at https://api.tsi-mlops.com/docs.

In [3]:
from dotenv import load_dotenv
import httpx
import os
import urllib3

In [4]:
load_dotenv()

API_KEY=os.getenv("API_KEY")
API_SECRET=os.getenv("API_SECRET")

In [7]:
# Read the test image file from local storage.
with open("testimg.png", "rb") as fp:
    file_data = fp.read()

## urrlib3

In [31]:
urllib_headers = urllib3.make_headers(basic_auth=f"{API_KEY}:{API_SECRET}")

fields = []

file_field = urllib3.fields.RequestField("file", file_data, filename="testimg.png")
file_field.make_multipart(content_disposition="form-data; name=testimg.png", content_type="image/png")
fields.append(file_field)

tags = ["test", "png", "urllib3"]

for tag in tags:
    tag_field = urllib3.fields.RequestField("tags", tag)
    tag_field.make_multipart(content_disposition="form-data; name=tags", content_type="text/plain")
    fields.append(tag_field)

urllib_upload_response = urllib3.request(
    "POST",
    "https://api.tsi-mlops.com/dataset/upload-file",
    fields=fields,
    headers=urllib_headers
)

if urllib_upload_response.status != 200:
    raise ValueError(f"Failed to upload file to S3: {urllib_upload_response.data}")

s3_object_name = urllib_upload_response.json()["s3_object_name"]

print(f"Uploaded file to S3 with object name: {s3_object_name}")


Uploaded file to S3 with object name: 1a8eef56-c585-4395-b2e4-a128ad90f1ed-testimg.png


In [32]:
urllib_list_response = urllib3.request(
    "GET",
    "https://api.tsi-mlops.com/dataset/list-files",
    headers=urllib_headers
)

urllib_lr_json = urllib_list_response.json()

urllib_files_list = urllib_lr_json["files"]

for file in urllib_files_list:
    print(f"File: {file['s3_object_name']}, Tags: {file['tags']}")

File: 210b3b04-fb3f-452c-8986-d00b83cce01e-testimg.png, Tags: []
File: 1c789734-9b0a-4897-a12f-a2033f1e035c-testimg.png, Tags: ['png']
File: 1a8eef56-c585-4395-b2e4-a128ad90f1ed-testimg.png, Tags: ['png', 'test', 'urllib3']
File: 7c1322ce-fa32-46fb-abe1-346a22fa42eb-testimg.png, Tags: ['test']
File: 59407ec5-1c27-4c09-b3fc-6721161b45b5-testimg.png, Tags: ['png', 'test']
File: c8c50c72-e8ba-4dea-8056-19c87283c523-testimg.png, Tags: ['test']
File: 8342ecaa-6d0c-4903-8d61-401481179946-testimg.png, Tags: ['png', 'test']
File: 3c4bc4c6-88e0-48f6-8476-8b04511af14c-testimg.png, Tags: ['png', 'test']
File: 0580bb15-17d7-46d9-a349-f34448b0661b-testimg.png, Tags: []


## httpx

In [33]:
httpx_upload_response = httpx.post(
    "https://api.tsi-mlops.com/dataset/upload-file",
    files={"file": ("testimg.png", file_data)},
    data={"tags": ["test", "png", "httpx"]},
    auth=(API_KEY, API_SECRET),
)

s3_object_name = httpx_upload_response.json()["s3_object_name"]

print(f"Uploaded file to S3 with object name: {s3_object_name}")

Uploaded file to S3 with object name: 9c6507a5-4606-44f9-aa18-826d9b1bc404-testimg.png


In [34]:
httpx_list_response = httpx.get(
    "https://api.tsi-mlops.com/dataset/list-files",
    auth=(API_KEY, API_SECRET),
)

httpx_lr_json = httpx_list_response.json()

httpx_files_list = httpx_lr_json["files"]

for file in httpx_files_list:
    print(f"File: {file['s3_object_name']}, Tags: {file['tags']}")

File: 210b3b04-fb3f-452c-8986-d00b83cce01e-testimg.png, Tags: []
File: 1c789734-9b0a-4897-a12f-a2033f1e035c-testimg.png, Tags: ['png']
File: 1a8eef56-c585-4395-b2e4-a128ad90f1ed-testimg.png, Tags: ['png', 'test', 'urllib3']
File: 7c1322ce-fa32-46fb-abe1-346a22fa42eb-testimg.png, Tags: ['test']
File: 9c6507a5-4606-44f9-aa18-826d9b1bc404-testimg.png, Tags: ['png', 'test', 'httpx']
File: 59407ec5-1c27-4c09-b3fc-6721161b45b5-testimg.png, Tags: ['png', 'test']
File: c8c50c72-e8ba-4dea-8056-19c87283c523-testimg.png, Tags: ['test']
File: 8342ecaa-6d0c-4903-8d61-401481179946-testimg.png, Tags: ['png', 'test']
File: 3c4bc4c6-88e0-48f6-8476-8b04511af14c-testimg.png, Tags: ['png', 'test']
File: 0580bb15-17d7-46d9-a349-f34448b0661b-testimg.png, Tags: []
